In [1]:
import os
import json
import torch
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

NVIDIA A100 80GB PCIe


In [2]:
for root, dirs, files in os.walk('/home/serfani/serfani_data0/E33OMA'):
    
    sorted_files = sorted(files)
    list1 = [os.path.join(root, file) for file in sorted_files if file.split(".")[1] == 'aijlh1E33oma_ai']   # Velocity Fields (time, level, lat, lon)
    list2 = [os.path.join(root, file) for file in sorted_files if file.split(".")[1] == 'cijh1E33oma_ai']    # Precipitation (time, lat, lon)
    list3 = [os.path.join(root, file) for file in sorted_files if file.split(".")[1] == 'taijh1E33oma_ai']   # Sea Salt Src (time, lat, lon)
    list4 = [os.path.join(root, file) for file in sorted_files if file.split(".")[1] == 'tNDaijh1E33oma_ai'] # Clay-BCB Src (time, lat, lon)
    list5 = [os.path.join(root, file) for file in sorted_files if file.split(".")[1] == 'taijlh1E33oma_ai']  # Aerosols Mixing Ratio (time, level, lat, lon)


In [3]:
data1 = list()
ds1  = xr.open_mfdataset(list1[:365]).isel(level=0)
data1.append([ds1['u'].mean().values.item(), ds1['u'].std().values.item(), ds1['u'].min().values.item(), ds1['u'].max().values.item()])
data1.append([ds1['v'].mean().values.item(), ds1['v'].std().values.item(), ds1['v'].min().values.item(), ds1['v'].max().values.item()])
data1.append([ds1['omega'].mean().values.item(), ds1['omega'].std().values.item(), ds1['omega'].min().values.item(), ds1['omega'].max().values.item()])

ds2  = xr.open_mfdataset(list2[:365])
data1.append([ds2['prec'].mean().values.item(), ds2['prec'].std().values.item(), ds2['prec'].min().values.item(), ds2['prec'].max().values.item()])

ds3  = xr.open_mfdataset(list3[:365])
data1.append([ds3['seasalt1_ocean_src'].mean().values.item(), ds3['seasalt1_ocean_src'].std().values.item(), ds3['seasalt1_ocean_src'].min().values.item(), ds3['seasalt1_ocean_src'].max().values.item()])

ds4  = xr.open_mfdataset(list4[:365])
data1.append([ds4['Clay_emission'].mean().values.item(), ds4['Clay_emission'].std().values.item(), ds4['Clay_emission'].min().values.item(), ds4['Clay_emission'].max().values.item()])
data1.append([ds4['BCB_biomass_src'].mean().values.item(), ds4['BCB_biomass_src'].std().values.item(), ds4['BCB_biomass_src'].min().values.item(), ds4['BCB_biomass_src'].max().values.item()])

ds5  = xr.open_mfdataset(list5[:365]).isel(level=0)
data1.append([ds5['seasalt1'].mean().values.item(), ds5['seasalt1'].std().values.item(), ds5['seasalt1'].min().values.item(), ds5['seasalt1'].max().values.item()])
data1.append([ds5['Clay'].mean().values.item(), ds5['Clay'].std().values.item(), ds5['Clay'].min().values.item(), ds5['Clay'].max().values.item()])
data1.append([ds5['BCB'].mean().values.item(), ds5['BCB'].std().values.item(), ds5['BCB'].min().values.item(), ds5['BCB'].max().values.item()])


In [4]:
data2 = list()
ds1  = xr.open_mfdataset(list1[:365]).isel(level=0)
u = np.array(np.ma.log10(ds1['u'].values))
data2.append([u.mean(), u.std(), u.min(), u.max()])

v = np.array(np.ma.log10(ds1['v'].values))
data2.append([v.mean(), v.std(), v.min(), v.max()])

w = np.array(np.ma.log10(ds1['omega'].values))
data2.append([w.mean(), w.std(), w.min(), w.max()])


ds2  = xr.open_mfdataset(list2[:365])
p = np.array(np.ma.log10(ds2['prec'].values))
data2.append([p.mean(), p.std(), p.min(), p.max()])


ds3  = xr.open_mfdataset(list3[:365])
ss_src = np.array(np.ma.log10(ds3['seasalt1_ocean_src'].values))
data2.append([ss_src.mean(), ss_src.std(), ss_src.min(), ss_src.max()])


ds4  = xr.open_mfdataset(list4[:365])
c_src = np.array(np.ma.log10(ds4['Clay_emission'].values))
data2.append([c_src.mean(), c_src.std(), c_src.min(), c_src.max()])

bc_src = np.array(np.ma.log10(ds4['BCB_biomass_src'].values))
data2.append([bc_src.mean(), bc_src.std(), bc_src.min(), bc_src.max()])


ds5  = xr.open_mfdataset(list5[:365]).isel(level=0)
ss_conc = np.array(np.ma.log10(ds5['seasalt1'].values))
data2.append([ss_conc.mean(), ss_conc.std(), ss_conc.min(), ss_conc.max()])

c_conc= np.array(np.ma.log10(ds5['Clay'].values))
data2.append([c_conc.mean(), c_conc.std(), c_conc.min(), c_conc.max()])

bc_conc = np.array(np.ma.log10(ds5['BCB'].values))
data2.append([bc_conc.mean(), bc_conc.std(), bc_conc.min(), bc_conc.max()])

In [10]:
import json

data = [data1, data2]

variables = ['u', 'v', 'w', 'prec', 'ss_src', 'c_src', 'bc_src', 'ss_conc', 'c_conc', 'bc_conc']

result = {'set1': {}, 'set2': {}}

for idx1, key in enumerate(result.keys()):
    for idx2, variable in enumerate(variables):
        mean, std, min_val, max_val = data[idx1][idx2]
        result[key][variable] = {
            'mean': float(mean),
            'std':  float(std),
            'min':  float(min_val),
            'max':  float(max_val)
        }

with open('variable_statistics.json', 'a') as jf:
    json.dump(result, jf, indent=4)


In [5]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message="Converting a CFTimeIndex with dates from a non-standard calendar")
    for root, dirs, files in os.walk('/home/serfani/serfani_data0/E33OMA'):
        
        sorted_files = sorted(files)
        list1 = [os.path.join(root, file) for file in sorted_files if file.split(".")[1] == 'aijlh1E33oma_ai']   # Velocity Fields (time, level, lat, lon)

    datetimeindex = xr.open_mfdataset(list1[:365]).indexes['time'].to_datetimeindex()

In [12]:
from torch.utils.data import Dataset
import torchvision.transforms as T
import warnings


class E33OMA(Dataset):

    def __init__(self, period, species, padding, root='/home/serfani/serfani_data0/E33OMA'):
        super(E33OMA, self).__init__()
        
        self.period  = period
        self.species = species
        self.padding = padding
        self.root    = root
        
        self._get_data_index()
    
    def _get_data_index(self):
        
        for root, dirs, files in os.walk(self.root):
            
            sorted_files = sorted(files)
            list1 = [os.path.join(root, file) for file in sorted_files if file.split(".")[1] == 'aijlh1E33oma_ai']   # Velocity Fields (time, level, lat, lon)
            # list2 = [os.path.join(root, file) for file in sorted_files if file.split(".")[1] == 'cijh1E33oma_ai']    # Precipitation (time, lat, lon)
            # list3 = [os.path.join(root, file) for file in sorted_files if file.split(".")[1] == 'taijh1E33oma_ai']   # Sea Salt Src (time, lat, lon)
            # list3 = [os.path.join(root, file) for file in sorted_files if file.split(".")[1] == 'tNDaijh1E33oma_ai'] # Clay-BCB Src (time, lat, lon)
            # list5 = [os.path.join(root, file) for file in sorted_files if file.split(".")[1] == 'taijlh1E33oma_ai']  # Aerosols Mixing Ratio (time, level, lat, lon)

        # Convert `cftime.DatetimeNoLeap` to `pandas.to_datetime()`
        warnings.filterwarnings("ignore", message="Converting a CFTimeIndex with dates from a non-standard calendar")
        datetimeindex = xr.open_mfdataset(list1[:365]).indexes['time'].to_datetimeindex()

        idx = np.arange(len(datetimeindex))
        rng = np.random.default_rng(0)
        rng.shuffle(idx)
        
        if   self.period == 'train':
            self.datetimeindex = datetimeindex[idx[:12264]] # 70%
        
        elif self.period == 'val':
            self.datetimeindex = datetimeindex[idx[12264:]] # 30%
        
        elif self.period == 'test':
            self.datetimeindex = xr.open_mfdataset(list1[365:]).indexes['time'].to_datetimeindex()

    def __getitem__(self, index):
        
        timestep = self.datetimeindex[index].strftime('%Y%m%d')
        
        ds1 = xr.open_dataset(os.path.join(self.root, f'{timestep}.aijlh1E33oma_ai.nc'))
        ds1['time'] = ds1.indexes['time'].to_datetimeindex()
        
        ds2 = xr.open_dataset(os.path.join(self.root, f'{timestep}.cijh1E33oma_ai.nc'))
        ds2['time'] = ds2.indexes['time'].to_datetimeindex()

        X1 = np.expand_dims(ds1['u'].isel(level=0).sel(time=self.datetimeindex[index]), axis=0)
        X2 = np.expand_dims(ds1['v'].isel(level=0).sel(time=self.datetimeindex[index]), axis=0)
        X3 = np.expand_dims(ds1['omega'].isel(level=0).sel(time=self.datetimeindex[index]), axis=0)

        X4 = np.expand_dims(ds2['prec'].sel(time=self.datetimeindex[index]), axis=0)

        with open('variable_statistics.json', 'r') as jf:
            vs = json.load(jf)
        
        X1_mean = vs['u']['mean'];    X1_std = vs['u']['std']
        X2_mean = vs['v']['mean'];    X2_std = vs['v']['std']
        X3_mean = vs['w']['mean'];    X3_std = vs['w']['std']
        X4_mean = vs['prec']['mean']; X4_std = vs['prec']['std']

        if self.species == 'seasalt':
            # Add positive lag for target variable
            ds3 = xr.open_dataset(os.path.join(self.root, f'{timestep}.taijh1E33oma_ai.nc'))
            ds3['time'] = ds3.indexes['time'].to_datetimeindex()

            ds4 = xr.open_dataset(os.path.join(self.root, f'{timestep}.taijlh1E33oma_ai.nc'))
            ds4['time'] = ds4.indexes['time'].to_datetimeindex()

            X5 = np.expand_dims(ds3['seasalt1_ocean_src'].sel(time=self.datetimeindex[index]), axis=0)
            y  = np.expand_dims(ds4['seasalt1'].isel(level=0).sel(time=self.datetimeindex[index]), axis=0)

            X5_mean = vs['ss_src']['mean']; X5_std = vs['ss_src']['std']
            y_mean  = vs['ss_conc']['mean']; y_std = vs['ss_conc']['std']

        if self.species == 'clay':
            # Add positive lag for target variable
            ds3 = xr.open_dataset(os.path.join(self.root, f'{timestep}.tNDaijh1E33oma_ai.nc'))
            ds3['time'] = ds3.indexes['time'].to_datetimeindex()

            ds4 = xr.open_dataset(os.path.join(self.root, f'{timestep}.taijlh1E33oma_ai.nc'))
            ds4['time'] = ds4.indexes['time'].to_datetimeindex()

            X5 = np.expand_dims(ds3['Clay_emission'].sel(time=self.datetimeindex[index]), axis=0)
            y  = np.expand_dims(ds4['Clay'].isel(level=0).sel(time=self.datetimeindex[index]), axis=0)

            X5_mean = vs['c_src']['mean']; X5_std = vs['c_src']['std']
            y_mean  = vs['c_conc']['mean']; y_std = vs['c_conc']['std']

        if self.species == 'bcb':
            # Add positive lag for target variable
            ds3 = xr.open_dataset(os.path.join(self.root, f'{timestep}.tNDaijh1E33oma_ai.nc'))
            ds3['time'] = ds3.indexes['time'].to_datetimeindex()

            ds4 = xr.open_dataset(os.path.join(self.root, f'{timestep}.taijlh1E33oma_ai.nc'))
            ds4['time'] = ds4.indexes['time'].to_datetimeindex()

            X5 = np.expand_dims(ds3['BCB_biomass_src'].sel(time=self.datetimeindex[index]), axis=0)
            y  = np.expand_dims(ds4['BCB'].isel(level=0).sel(time=self.datetimeindex[index]), axis=0)

            X5_mean = vs['bc_src']['mean']; X5_std = vs['bc_src']['std']
            y_mean  = vs['bc_conc']['mean']; y_std = vs['bc_conc']['std']


        X = np.concatenate((X1, X2, X3, X4, X5), axis=0)  # (5, 90, 144)

        Xs_mean = np.array((X1_mean, X2_mean, X3_mean, X4_mean, X5_mean), dtype=np.float32).reshape(-1, 1, 1)
        Xs_std  = np.array((X1_std, X2_std, X3_std, X4_std, X5_std), dtype=np.float32).reshape(-1, 1, 1)
 
        self.y_mean = np.array(y_mean, dtype=np.float32).reshape(-1, 1, 1)
        self.y_std  = np.array(y_std, dtype=np.float32).reshape(-1, 1, 1)

        X = (X - Xs_mean) / Xs_std
        y = (y -  self.y_mean) / self.y_std

        if self.padding:
            w = X.shape[2] # width
            h = X.shape[1] # height
            
            top_pad   = self.padding - h
            right_pad = self.padding - w
            
            X = np.lib.pad(X, ((0, 0), (top_pad, 0), (0, right_pad)), mode='constant', constant_values=0)
        
        X = torch.from_numpy(X).type(torch.float32) # torch image: C x H x W
        y = torch.from_numpy(y).type(torch.float32) # torch image: C x H x W

        return X, y
        
    def __len__(self):
        return len(self.datetimeindex)

In [13]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message="Converting a CFTimeIndex with dates from a non-standard calendar")
    
    dataset = E33OMA(period='test', species='bcb', padding=256)

    print(len(dataset))
    dataiter = iter(dataset)
    X, y = next(dataiter)
    print(X.shape, y.shape)

17520
torch.Size([5, 256, 256]) torch.Size([1, 90, 144])
